In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# pass in column names for each CSV as the column name is not given in the file and read them using pandas.
# You can check the column names from the readme file

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,
encoding='latin-1')

In [2]:
print(users.shape)
users.head()

(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [3]:
print(ratings.shape)
ratings.head()

(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
print(items.shape)
items.head()

(1682, 24)


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# Load Train and Test datasets
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [6]:
# Building collaborative filtering model from scratch

'''
We will recommend movies based on user-user similarity and 
item-item similarity. For that, first we need to calculate 
the number of unique users and movies.
'''

n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]

In [8]:
'''
Now, we will create a user-item matrix which can be used 
to calculate the similarity between users and items.
'''
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [11]:
'''Now, we will calculate the similarity. We can use the 
pairwise_distance function from sklearn to calculate the cosine similarity.
'''
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [15]:
'''
This gives us the item-item and user-user similarity in an array form. 
The next step is to make predictions based on these similarities. 
Let’s define a function to do just that.
'''
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred


In [16]:
'''Finally, we will make predictions based on user similarity and item similarity.'''
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

In [29]:
pd.DataFrame(user_prediction)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.065326,0.734303,0.629924,1.010669,0.640686,0.476150,1.784569,1.163032,1.513350,0.704478,...,0.394041,0.394434,0.393981,0.392972,0.393344,0.392272,0.394909,0.393590,0.393049,0.392771
1,1.763088,0.384040,0.196179,0.731538,0.225643,0.003892,1.493597,0.876153,1.108467,0.261991,...,-0.086942,-0.085491,-0.087137,-0.088158,-0.087298,-0.089288,-0.087468,-0.088378,-0.086918,-0.086712
2,1.795904,0.329047,0.158829,0.684154,0.173277,-0.035621,1.488230,0.835769,1.135426,0.236383,...,-0.134795,-0.133537,-0.135543,-0.136438,-0.135041,-0.137611,-0.136374,-0.136992,-0.134969,-0.134765
3,1.729951,0.293913,0.127741,0.644932,0.142143,-0.062261,1.437010,0.796249,1.096663,0.211789,...,-0.161413,-0.160220,-0.161542,-0.162586,-0.161634,-0.163877,-0.162283,-0.163080,-0.161442,-0.161248
4,1.796651,0.454474,0.354422,0.763130,0.359539,0.195987,1.547370,0.908904,1.292027,0.437954,...,0.101762,0.102405,0.101923,0.100839,0.101711,0.099951,0.102515,0.101233,0.101075,0.101201
5,1.996889,0.651172,0.528276,0.921054,0.537388,0.343124,1.722899,1.030714,1.375829,0.570538,...,0.261308,0.261053,0.261749,0.260694,0.260574,0.259848,0.262375,0.261111,0.261061,0.261089
6,2.466055,1.099186,1.016489,1.362861,1.006458,0.853382,2.180302,1.498903,1.891000,1.072978,...,0.765553,0.765650,0.766179,0.765129,0.765580,0.764340,0.766967,0.765654,0.764983,0.765165
7,1.693486,0.296285,0.195398,0.612075,0.198638,0.025983,1.400383,0.778876,1.136172,0.278078,...,-0.070712,-0.070081,-0.071161,-0.072135,-0.071083,-0.072866,-0.070435,-0.071651,-0.071744,-0.071357
8,1.681165,0.279528,0.121887,0.616094,0.136368,-0.075169,1.384177,0.755221,1.050439,0.191584,...,-0.161335,-0.160330,-0.161420,-0.162461,-0.161416,-0.163505,-0.161425,-0.162465,-0.161396,-0.161430
9,2.021688,0.650911,0.530669,0.916215,0.531720,0.348246,1.722691,1.048935,1.389699,0.578968,...,0.264904,0.264997,0.265291,0.264285,0.264545,0.263527,0.266037,0.264782,0.264589,0.264781


In [23]:

# def get_user_prediction()

user_pred = pd.DataFrame(user_prediction)[:1].T
user_pred['movie'] = user_pred.index
user_pred.columns = ['similarity', 'movie']


user_pred.nlargest(10, columns='similarity').tail(10)

,similarity,movie
49,2.820170,49
99,2.427339,99
257,2.419308,257
180,2.316165,180
285,2.296358,285
287,2.151160,287
299,2.129948,299
126,2.079910,126
0,2.065326,0
293,2.029749,293


In [27]:
ratings[ratings['user_id'] == 1].head(5)

,user_id,movie_id,rating,unix_timestamp
202,1,61,4,878542420
305,1,189,3,888732928
333,1,33,4,878542699
334,1,160,4,875072547
478,1,20,4,887431883


In [ ]:
# Building a simple popularity and collaborative filtering model using Turicreate

'''
After installing turicreate, first let’s import it and 
read the train and test dataset in our environment. 
Since we will be using turicreate, we will need to convert the dataset in SFrames.
'''
import turicreate
train_data = turicreate.SFrame(ratings_train)
test_data = turicreate.SFrame(ratings_test)

In [ ]:
'''
We have user behavior as well as attributes of the users and movies, 
so we can make content based as well as collaborative filtering algorithms. 
We will start with a simple popularity model and then build a collaborative filtering model.

First we’ll build a model which will recommend movies based on the 
most popular choices, i.e., a model where all the users receive the 
same recommendation(s). We will use the turicreate recommender
function popularity_recommender for this.
'''

popularity_model = turicreate.popularity_recommender.create(train_data,
                                                            user_id='user_id',
                                                            item_id='movie_id',
                                                            target='rating')

In [ ]:
'''It’s prediction time! We will recommend the top 5 items for the first 5 users in our dataset.'''

popularity_recomm = popularity_model.recommend(users=[1,2,3,4,5],k=5)

popularity_recomm.print_rows(num_rows=25)

In [ ]:
'''Note that the recommendations for all users are the same – 1467, 1201, 1189, 1122, 814.
And they’re all in the same order! This confirms that all the recommended movies 
have an average rating of 5, i.e. all the users who watched the movie gave it a top 
rating. Thus our popularity system works as expected.

After building a popularity model, we will now build a collaborative 
filtering model. Let’s train the item similarity model and make top 5 
recommendations for the first 5 users.
'''

#Training the model
item_sim_model = turicreate.item_similarity_recommender.create(train_data,
                                                               user_id='user_id',
                                                               item_id='movie_id',
                                                               target='rating',
                                                               similarity_type='cosine')

#Making recommendations
item_sim_recomm = item_sim_model.recommend(users=[1,2,3,4,5],k=5)

item_sim_recomm.print_rows(num_rows=25)

In [ ]:
'''Here we can see that the recommendations (movie_id) are different 
for each user. So personalization exists, i.e. for different users 
we have a different set of recommendations.

In this model, we do not have the ratings for each movie 
given by each user. We must find a way to predict all these missing 
ratings. For that, we have to find a set of features which can 
define how a user rates the movies. These are called latent features. 
We need to find a way to extract the most important latent features 
from the the existing features. Matrix factorization, covered in the 
next section, is one such technique which uses the lower dimension 
dense matrix and helps in extracting the important latent features.'''


In [ ]:
# Building a recommendation engine using matrix factorization

class MF():
    '''
    R – The user-movie rating matrix
    K – Number of latent features
    alpha – Learning rate for stochastic gradient descent
    beta – Regularization parameter for bias
    iterations – Number of iterations to perform stochastic gradient descent
    '''

    # Initializing the user-movie rating matrix, no. of latent features, alpha and beta.
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    # Initializing user-feature and movie-feature matrix 
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
        for j in range(self.num_items)
        if self.R[i, j] > 0
        ]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    # Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Ratings for user i and moive j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [ ]:
'''We have to convert the user item ratings to matrix form. It can be done using the pivot function in python.'''
R= np.array(ratings.pivot(index = 'user_id',
                          columns ='movie_id',
                          values = 'rating').fillna(0))

In [ ]:
'''
fillna(0) will fill all the missing ratings with 0. Now we have the R matrix. 
We can initialize the number of latent features, but the number of these 
features must be less than or equal to the number of original features.

Now let us predict all the missing ratings. 

Let’s take K=20, alpha=0.001, beta=0.01 and iterations=100.
'''
mf = MF(R, K=20, alpha=0.001, beta=0.01, iterations=100)

training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

In [ ]:
popularity_model

In [ ]:
model_performance = turicreate.recommender.util.compare_models(test_data, [popularity_model, item_sim_model])